In [1]:
from modes import *
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Concatenate, Reshape

In [2]:
input_layer_1d = Input(shape=(3, ))
input_layer_2d = Input(shape=(201, 201, 4))

conv1 = Conv2D(16, kernel_size=(5, 5), activation='relu')(input_layer_2d)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = Conv2D(32, kernel_size=(3, 3), activation='relu')(pool1)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
conv3 = Conv2D(32, kernel_size=(3, 3), activation='relu')(pool2)
flat1 = Flatten()(conv3)
merge = Concatenate()([flat1, input_layer_1d])
flat2 = Flatten()(merge)
dense1 = Dense(64, activation='tanh')(flat2)
dense2 = Dense(128, activation='relu')(dense1)
dense3 = Dense(200 * 200, activation='sigmoid')(dense2)
output = Reshape((200, 200))(dense3)

model = Model(inputs=[input_layer_1d, input_layer_2d], outputs=output)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 201, 201, 4  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 197, 197, 16  1616        ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 98, 98, 16)   0           ['conv2d[0][0]']                 
                                                                                              

In [2]:
def load_data(bc_code, load_code, penal, r, vf):
    bc_code, load_code, penal, r, vf = [param.numpy() for param in (bc_code, load_code, penal, r, vf)]
    bc_code, load_code = bc_code.decode('utf-8'), load_code.decode('utf-8')
    load_path = 'data/' + bc_code + '/' + load_code + '/'
    code = f'{bc_code}-{load_code}-P{penal}-R{r:0.3f}-V{vf:0.3f}'

    c = np.load(load_path + bc_code + load_code + '-C.npy')
    ux = np.load(load_path + bc_code + load_code + '-X.npy')
    uy = np.load(load_path + bc_code + load_code + '-Y.npy')

    input_2d = np.ones((201, 201, 4), dtype=np.float32)
    input_2d[:, :, 0] = ux
    input_2d[:, :, 1] = uy
    input_2d[:, :, 2] = np.hypot(ux, uy)
    input_2d[:, :, 3] = np.pad(c, pad_width=1, mode='edge')[0:-1, 0:-1]

    y = np.load(load_path + 'output/' + code + '.npy').astype(np.float32)

    return np.array((penal, r, vf)), input_2d, y

In [3]:
def general_function(bc_code, load_code, penal, r, vf):
    input_1d, input_2d, y = tf.py_function(load_data, (bc_code, load_code, penal, r, vf),
                                           (tf.float32, tf.float32, tf.float32))
    input_1d.set_shape(tf.TensorShape([3, ]))
    input_2d.set_shape(tf.TensorShape([201, 201, 4]))
    y.set_shape(tf.TensorShape([200, 200]))
    return (input_1d, input_2d), y

In [4]:
parameters = [(b, l, p, r, v) for b in list(bcs.keys()) for l in list(loads.keys())
              for p in ps for r in rs for v in vfs]
np.random.shuffle(parameters)
parameters = tuple(np.array(item) for item in zip(*parameters))

In [5]:
def filter_function(bc_code, load_code, penal, r, vf):
    bc_code, load_code, penal, r, vf = [param.numpy() for param in (bc_code, load_code, penal, r, vf)]
    bc_code, load_code = bc_code.decode('utf-8'), load_code.decode('utf-8')
    load_path = 'data/' + bc_code + '/' + load_code + '/'
    code = f'{bc_code}-{load_code}-P{penal}-R{r:0.3f}-V{vf:0.3f}'
    y_path = load_path + 'output/' + code + '.npy'
    c_path = load_path + bc_code + load_code + '-C.npy'
    return os.path.exists(y_path) and np.load(c_path).any()

In [6]:
dataset = tf.data.Dataset.from_tensor_slices(parameters)
dataset = dataset.filter(lambda b, l, p, r, v: tf.py_function(filter_function, (b, l, p, r, v), tf.bool))
dataset = dataset.map(lambda b, l, p, r, v: general_function(b, l, p, r, v))
dataset = dataset.batch(25)

In [8]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.fit(dataset, epochs=20, batch_size=25)

Epoch 1/20
    378/Unknown - 385s 1s/step - loss: 0.2295 - accuracy: 0.0042

KeyboardInterrupt: 

In [8]:
batch = next(dataset.as_numpy_iterator())
x = batch[0][1][0]
c = x[:, :, 3]
type(c)

numpy.ndarray

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(c)
plt.show()

In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0
